# Machine Learning Beispiel mit AutoML (PyCaret)

In diesem Notebook wird mittels einem AutoML-Modell eine Pipeline durchlaufen um ein Modell für Vorhersagen zu trainieren. Dafür wird die Python-Bibliothek [PyCaret](https://pycaret.org/) genutzt. In diesem AutoML-Modell werden weitestgehend alle Vorverarbeitungsprozesse und auch verschiedene Modelle automatisch genutzt um das "bestmögliche" mit Hyperparametern angepasste Modell zu trainieren. Dieses Modell kann heruntergeladen und für Vorhersagen genutzt werden.

All diese Schritte werden hier in Folge gezeigt.



1.   Installation aller nötigen Bibliotheken
2.   Import der Funktionen der entsprechenden Bibliotheken
3.   Datensatz beziehen (hochladen oder online-Link)
4.   Datensatz analysieren mit Y-Data
5.   Datensatz vorverarbeiten
6.   Auswahl der Merkmale
7.   Auswahl des Vorhersageproblems (Klassifikation/Regression)
8.   Evaluierung des Modells
9.   Export des Modells
10.  Vorhersage auf unbekannte Daten



# 1. Installation der relevanten Bibliotheken
Es werden zunächst alle relevanten Bibliotheken installiert. In Colab muss dafür vor dem pip install ein ! stehen. Dies für die Ausführung auf dem eigenen Rechner (z.B. in VisualStudioCode) entfernen.
%%capture bedeutet, dass die Installationsprozesse nicht in der Ausgabe des Codes angezeigt werden.

In [27]:
#Installation aller nötigen Python-Bibliotheken um ein AutoML auf einen Datensatz anzuwenden
%%capture
!pip install -q plotly pycaret pandas ydata-profiling imblearn lightgbm

print("Alles Fertig!")

# 2. Import aller relevanten Bibliotheken

Es werden alle nötigen Bibliotheken und teils nur Funktionen dieser Bibliotheken (um effizienterweise Speicherplatz zu sparen) importiert.

In [28]:
# Importiere alle nötigen Bibliotheken
from google.colab import files
import os
import pandas as pd
from ydata_profiling import ProfileReport
from IPython.display import display, HTML, FileLink, clear_output
import ipywidgets as widgets
import urllib.request
from pycaret.datasets import get_data
from pycaret.regression import *
from pycaret.classification import *
from imblearn.over_sampling import RandomOverSampler, SMOTE
import lightgbm as lgb

## 3. Datensatz beziehen

In [29]:
# Überprüfen Sie, ob die Datei existiert und löschen Sie sie, falls sie existiert
if os.path.exists("dataset.csv"):
  os.remove("dataset.csv")

# Frage den Benutzer, ob er einen Datensatz hochladen möchte oder einen Datensatz aus einer Onlinequelle verwenden möchte
source = input("Möchten Sie einen Datensatz aus einer Onlinequelle verwenden oder einen Datensatz hochladen? (o/h)")

# Wenn der Benutzer einen Datensatz aus einer Onlinequelle verwenden möchte, frage nach der Quelle
if source == "o":
    online_source = input("Welche Onlinequelle möchten Sie verwenden? (UCI Machine Learning Repository/Kaggle)")
    urllib.request.urlretrieve(online_source, 'dataset.csv')
    df = pd.read_csv('dataset.csv')

# Wenn der Benutzer einen Datensatz hochladen möchte, füge eine Upload-Funktion für den Datensatz hinzu
if source == "h":
    # Wähle die Datei aus, die du hochladen möchtest
    uploaded = files.upload()
    # Ändere den Dateinamen
    for fn in uploaded.keys():
        os.rename(fn, 'dataset.csv')
    # Öffne die Datei
    df = pd.read_csv("dataset.csv")

Möchten Sie einen Datensatz aus einer Onlinequelle verwenden oder einen Datensatz hochladen? (o/h)o
Welche Onlinequelle möchten Sie verwenden? (UCI Machine Learning Repository/Kaggle)https://raw.githubusercontent.com/ProfEngel/datasets/main/Schwertlilie.csv


# 5. Datensatz analysieren mit YData

Mit der Bibliothek [YData-Profiling](https://docs.profiling.ydata.ai/4.6/) können sehr schnell und einfach statistische Kennzahlen eines Datensatzes ausgegeben werden. Dadurch können im Vorfeld erste Erkenntnisse über mögliche Korrelationen, Ausreisser und weiteres ermittelt werden.

In [30]:
# Erstelle den Profilbericht
profile = ProfileReport(df, title="Auswertung des Datensatz in einem Profiling Report")
profile.to_file("output.html")

# Zeige den Bericht im Notebook an
display(HTML("output.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Number of variables,5
Number of observations,150
Missing cells,0
Missing cells (%),0.0%
Duplicate rows,2
Duplicate rows (%),1.3%
Total size in memory,6.0 KiB
Average record size in memory,40.9 B
Numeric,4
Categorical,1
Dataset has 2 (1.3%) duplicate rows,Duplicates


# 6. Merkmalsauswahl
Auswahl des Targets und welche Merkmale in der Untersuchung berücksichtigt werden sollen.

In [31]:
# Lassen Sie den Benutzer das Target-Merkmal auswählen
print("Verfügbare Merkmale:", df.columns.tolist())
target_column = input("Welches Merkmal soll das Target sein? ")

# Zeilen mit fehlenden Werten in der Ziel-Spalte entfernen
df = df.dropna(subset=[target_column])

# Lassen Sie den Benutzer die Merkmale auswählen, die in der Vorverarbeitung berücksichtigt werden sollen
print("\nWählen Sie die Merkmale aus, die Sie in der Vorverarbeitung berücksichtigen möchten:")
features_to_include = {}
for col in df.columns:
    if col != target_column:
        include = input(f"Möchten Sie das Merkmal '{col}' berücksichtigen? (ja/nein) ").lower() == 'ja'
        features_to_include[col] = include

# Filtern Sie die Daten basierend auf den Auswahlkriterien des Benutzers
selected_features = [col for col, include in features_to_include.items() if include]
df = df[selected_features + [target_column]]

# Zeige die ersten Zeilen des DataFrames
display(df.head())

Verfügbare Merkmale: ['Länge des Kelchblatts', 'Kelchblattweite', 'Blütenblattlänge', 'Blütenblattweite', 'Klasse']
Welches Merkmal soll das Target sein? Klasse

Wählen Sie die Merkmale aus, die Sie in der Vorverarbeitung berücksichtigen möchten:
Möchten Sie das Merkmal 'Länge des Kelchblatts' berücksichtigen? (ja/nein) ja
Möchten Sie das Merkmal 'Kelchblattweite' berücksichtigen? (ja/nein) ja
Möchten Sie das Merkmal 'Blütenblattlänge' berücksichtigen? (ja/nein) ja
Möchten Sie das Merkmal 'Blütenblattweite' berücksichtigen? (ja/nein) ja


,Länge des Kelchblatts,Kelchblattweite,Blütenblattlänge,Blütenblattweite,Klasse
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Vorverarbeitungskonfiguration für AutoML-Modell
Dieser Abschnitt des Notebooks erlaubt die Konfiguration der Datenvorverarbeitung für ein AutoML-Modell. Die Vorverarbeitung kann entweder automatisch oder manuell durchgeführt werden, basierend auf der Entscheidung des Benutzers.

**Automatische vs. Manuelle Vorverarbeitung:**

Wenn das Zielattribut (target_column) numerisch ist, werden die Vorverarbeitungsparameter automatisch gesetzt.
Andernfalls wird der Benutzer gefragt, ob er die Vorverarbeitung automatisch oder manuell durchführen möchte.
Automatische Vorverarbeitung:

Bei Auswahl von "Automatisch" werden Standard-Vorverarbeitungsschritte angewandt, darunter:

*   Einfache Imputation (simple imputation) fehlender Werte.
*   Normalisierung der Daten.
*   Entfernung von Multikollinearität mit einem Schwellenwert von 0.95.
*   Hinzufügen polynomialer Merkmale.
*   Durchführung von Merkmalsauswahl (Feature Selection).
*   Principal Component Analysis (PCA), falls die Anzahl der Merkmale (ohne das Zielmerkmal) größer als 10 ist.

**Manuelle Vorverarbeitung:**

Bei manueller Auswahl können die Vorverarbeitungsschritte individuell festgelegt werden, einschließlich:
*   Art der Imputation (simple/iterative).
*   Entscheidung über Normalisierung.
*   Entfernung von Multikollinearität und Festlegung des Schwellenwerts.
*   Hinzufügen polynomialer Merkmale.
*   Durchführung von Merkmalsauswahl.
*   Entscheidung über die Anwendung von PCA, falls die Anzahl der Merkmale (ohne das Zielmerkmal) größer als 10 ist.

**Gemeinsame Parameter:**

Unabhängig von der Wahl der Vorverarbeitung werden die Daten (df), das Zielmerkmal (target_column) und eine feste session_id als Parameter gesetzt.

**Einfache Imputation fehlender Werte:**

Ziel: Behandlung von fehlenden Daten in Ihrem Dataset.
Nutzen: Ermöglicht dem Modell, auch mit unvollständigen Daten umzugehen, indem es fehlende Werte durch plausible Schätzungen ersetzt (z.B. Durchschnittswerte).

**Normalisierung der Daten:**

Ziel: Skalierung der Daten, so dass sie innerhalb eines bestimmten Bereichs (typischerweise zwischen 0 und 1) liegen.
Nutzen: Verhindert, dass Merkmale mit größeren Maßstäben die Modellierung dominieren und verbessert die Konvergenzgeschwindigkeit bei der Optimierung.

**Entfernung von Multikollinearität mit einem Schwellenwert von 0.95:**

Ziel: Identifikation und Entfernung von Merkmalen, die stark miteinander korreliert sind.
Nutzen: Vermeidet Redundanzen und Stabilitätsprobleme im Modell, da hochkorrelierte Merkmale das Modell und dessen Interpretation verzerren können.

**Hinzufügen polynomialer Merkmale:**

Ziel: Erzeugung neuer Merkmale, die polynomiale Kombinationen der ursprünglichen Merkmale sind.
Nutzen: Ermöglicht es dem Modell, komplexere Beziehungen in den Daten zu erfassen und zu lernen, was besonders bei nichtlinearen Zusammenhängen nützlich ist.

**Durchführung von Merkmalsauswahl (Feature Selection):**

Ziel: Auswahl der relevantesten Merkmale für das Modell.
Nutzen: Reduziert die Dimensionalität der Daten, verbessert die Modellleistung und vereinfacht das Modell, indem irrelevante oder redundante Merkmale entfernt werden.

**Principal Component Analysis (PCA), falls die Anzahl der Merkmale größer als 10 ist:**

Ziel: Reduktion der Dimensionalität der Daten durch Transformation in einen neuen Merkmalsraum.
Nutzen: Identifiziert die wichtigsten Merkmale, die die meiste Varianz in den Daten erklären, und reduziert die Komplexität des Modells, ohne signifikante Informationsverluste zu erleiden.

In [32]:
# Lassen Sie den Benutzer zwischen automatischer und manueller Vorverarbeitung wählen
if isinstance(df[target_column].iloc[0], (int, float)):
        preprocessing_params = {
            'data': df,
            'target': target_column,
            'session_id': 42,
        }
else:
    preprocessing_choice = input("Möchten Sie die Vorverarbeitung automatisch durchführen lassen oder selbst auswählen? (Automatisch/Manuell): ")

    if preprocessing_choice.lower() == "automatisch":
        # Prüfen, ob die Anzahl der Merkmale (ohne das Zielmerkmal) größer als 10 ist
        pca = True if len(df.columns) - 1 > 10 else False
        if pca:
            pca_choice = input("Principal Component Analysis (PCA) durchführen? (ja/nein): ").lower() == 'ja'
            pca = pca_choice

        preprocessing_params = {
            'data': df,
            'target': target_column,
            'session_id': 42,
            'imputation_type': 'simple',
            'normalize': False,
            'remove_multicollinearity': False,
            'multicollinearity_threshold': 0.95,
            'polynomial_features': False,
            'feature_selection': False,
            'pca': pca
        }

    else:
        # Manuelle Auswahl der Vorverarbeitungsschritte
        print("\nWählen Sie die gewünschten Vorverarbeitungsschritte:")
        imputation_type = input("Art der Imputation (simple/iterative): ")
        normalize = input("Daten normalisieren? (ja/nein): ").lower() == 'ja'
        remove_multicollinearity = input("Multikollinearität entfernen? (ja/nein): ").lower() == 'ja'
        multicollinearity_threshold = float(input("Schwellenwert für Multikollinearität (0.0 bis 1.0): "))
        polynomial_features = input("Polynomiale Merkmale hinzufügen? (ja/nein): ").lower() == 'ja'
        feature_selection = input("Merkmal-Auswahl durchführen? (ja/nein): ").lower() == 'ja'

        # Prüfen, ob die Anzahl der Merkmale (ohne das Zielmerkmal) größer als 10 ist
        pca = True if len(df.columns) - 1 > 10 else False
        if pca:
            pca_choice = input("Principal Component Analysis (PCA) durchführen? (ja/nein): ").lower() == 'ja'
            pca = pca_choice

        preprocessing_params = {
            'data': df,
            'target': target_column,
            'session_id': 42,
            'imputation_type': imputation_type,
            'normalize': normalize,
            'remove_multicollinearity': remove_multicollinearity,
            'multicollinearity_threshold': multicollinearity_threshold,
            'polynomial_features': polynomial_features,
            'feature_selection': feature_selection,
            'pca': pca
        }

Möchten Sie die Vorverarbeitung automatisch durchführen lassen oder selbst auswählen? (Automatisch/Manuell): Automatisch


# Wahl der Vorhersageart (Klassifikation oder Regression)

**Überprüfung des Zielmerkmals:**

Der Code überprüft, ob ***target_column*** definiert und nicht None ist.
Basierend auf dem Datentyp des Zielmerkmals (numerisch oder kategorisch) wird zwischen Regression und Klassifikation unterschieden.

**Regressionstraining (für kontinuierliche Zielmerkmale):**

Nach Bestätigung durch den Benutzer beginnt das Regressionstraining.
Der Prozess umfasst:


*   Initialisierung des Regressionsmodells mit vordefinierten Vorverarbeitungsparametern.
*   Vergleich verschiedener Modelle.
*   Auswertung des besten Modells.
*   Anzeige statistischer Zusammenfassungen und Vorhersagen.
*   Speicherung des besten Modells.

**Klassifikationstraining (für kategorische Zielmerkmale):**

Beinhaltet die Untersuchung der Klassenverteilung und Anwendung von Datenausgleichsmethoden (z.B. SMOTE oder RandomOverSampler).
Nach Benutzerbestätigung erfolgt das Klassifikationstraining.
Der Prozess umfasst:
*   Initialisierung des Klassifikationsmodells mit den angepassten Vorverarbeitungsparametern.
*   Auswahl des Modells basierend auf einem vom Benutzer gewählten Kriterium (z.B. AUC, Genauigkeit).
*   Vergleich und Bewertung des besten Modells.
*   Anzeige von Feature Importance, falls verfügbar.
*   Vorhersagen und Speicherung des besten Modells.

**Gemeinsame Schritte:**

Beide Prozesse (Regression und Klassifikation) umfassen das Anzeigen der ersten Zeilen des DataFrames, die statistische Zusammenfassung, die Vorhersagen des Modells und das Speichern des besten Modells.

In [33]:
# Überprüfen Sie, ob target_column definiert ist und nicht None ist
if target_column is not None:

    # Bestimmen Sie, ob das Zielmerkmal numerisch oder kategorisch ist
    if isinstance(df[target_column].iloc[0], (int, float)):
        # Wenn das Zielmerkmal kontinuierlich ist, verwende das pycaret.regression Modul

        choice = input("Möchten Sie das Regressionstraining starten? (ja/nein): ").lower()
        if choice == 'ja':
            ############################
            # Regression klappt nicht... Hier ein paar Erklärungsversuche
            #if df[target_column].isnull().any():
              #df.dropna(subset=[target_column], inplace=True)
            #if df[target_column].nunique() < 10:
              #print(f"Warnung: Das Zielattribut hat nur {df[target_column].nunique()} eindeutige Werte. Ist dies sicherlich ein Regressionsproblem?")
            #df[target_column] = df[target_column].astype(float)
            #print(df[target_column].value_counts())
            #reg = setup(data = df, target = target_column, session_id=123)
            ############################
            reg = setup(**preprocessing_params)
            setup_df = pull()
            print("\nDies ist das AutoML Training")
            display(setup_df)
            best_model = compare_models()
            compare_df = pull()
            print("\nHier sind die AutoML Modelle")
            display(compare_df)
            print(f"\nBestes Modell: {best_model}")

            # Diagramme für Regression
            evaluate_model(best_model)

            # Zeige die ersten Zeilen des DataFrames
            display(df.head())

            # Statistische Zusammenfassung
            display(df.describe())

            # Modell vorhersagen
            predictions = predict_model(best_model)
            print("\nVorhersagen des Modells:")
            display(predictions)

            # Modell speichern
            save_model(best_model, 'best_model')
            print("\nDas Modell wurde als 'best_model' gespeichert.")

    else:
        # Wenn das Zielmerkmal kategorisch ist, verwende das pycaret.classification Modul

        # Prüfen Sie die Anzahl der Beispiele in jeder Klasse
        class_counts = df[target_column].value_counts()

        # Prüfen Sie die Anzahl der Beispiele in der kleinsten Klasse
        min_class_count = class_counts.min()

        # Wenn die kleinste Klasse weniger als 6 Beispiele hat (standardmäßige n_neighbors für SMOTE + 1),
        # verwenden Sie ROS, andernfalls verwenden Sie SMOTE.
        if min_class_count < 6:
            resampling_method = RandomOverSampler()
        else:
            resampling_method = SMOTE()

        preprocessing_params['fix_imbalance'] = True
        preprocessing_params['fix_imbalance_method'] = resampling_method

        # Benutzer wählt das Kriterium aus
        metrics = {
            "AUC": "AUC (Area Under the Curve)",
            "Accuracy": "Accuracy",
            "Recall": "Recall",
            "Precision": "Precision",
            "F1": "F1 Score"
        }
        sort_metric = input(f"Wählen Sie das Kriterium für die Modellauswahl aus {list(metrics.keys())}: ")
        print(metrics[sort_metric])

        choice = input("Möchten Sie das Klassifikationstraining starten? (ja/nein): ").lower()
        if choice == 'ja':
            clf = setup(**preprocessing_params)
            setup_df = pull()
            print("\nDies ist das AutoML Training")
            display(setup_df)

            best_model = compare_models()
            print(f"\nBestes Modell basierend auf {sort_metric}: {best_model}")

            compare_df = pull()
            print("\nDies ist das ML Modell")
            display(compare_df)

            # Diagramme für Klassifikation
            evaluate_model(best_model)

            # Überprüfen, ob das Modell coef_ oder feature_importances_ Attribute hat
            if hasattr(best_model, 'coef_') or hasattr(best_model, 'feature_importances_'):
                print("\nFeature Importance:")
                plot_model(best_model, plot='feature')
            else:
                print("\nFeature Importance ist für dieses Modell nicht verfügbar.")

            # Zeige die ersten Zeilen des DataFrames
            display(df.head())

            # Statistische Zusammenfassung
            display(df.describe())

            # Modell vorhersagen
            if isinstance(best_model, list):
                if len(best_model) > 0:
                    best_model = best_model[0]
                    predictions = predict_model(best_model)
                    print("\nVorhersagen des Modells:")
                    display(predictions)
                else:
                    print("Error: Kein Modell in der Liste 'best_model' gefunden.")
                    # Beenden Sie hier, da kein Modell zum Vorhersagen vorhanden ist

            # Modell speichern
            save_model(best_model, 'best_model')
            print("\nDas Modell wurde als 'best_model' gespeichert.")

Wählen Sie das Kriterium für die Modellauswahl aus ['AUC', 'Accuracy', 'Recall', 'Precision', 'F1']: Accuracy
Accuracy
Möchten Sie das Klassifikationstraining starten? (ja/nein): ja


,Description,Value
0,Session id,42
1,Target,Klasse
2,Target type,Multiclass
3,Target mapping,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virginica: 2"
4,Original data shape,"(150, 5)"
5,Transformed data shape,"(150, 5)"
6,Transformed train set shape,"(105, 5)"
7,Transformed test set shape,"(45, 5)"
8,Numeric features,4
9,Preprocess,True



Dies ist das AutoML Training


,Description,Value
0,Session id,42
1,Target,Klasse
2,Target type,Multiclass
3,Target mapping,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virgi..."
4,Original data shape,"(150, 5)"
5,Transformed data shape,"(150, 5)"
6,Transformed train set shape,"(105, 5)"
7,Transformed test set shape,"(45, 5)"
8,Numeric features,4
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.9809,1.0000,0.9809,0.9852,0.9805,0.9711,0.9735,0.0410
nb,Naive Bayes,0.9727,0.9974,0.9727,0.9806,0.9714,0.9588,0.9638,0.0850
lda,Linear Discriminant Analysis,0.9718,0.9974,0.9718,0.9780,0.9712,0.9573,0.9609,0.0450
et,Extra Trees Classifier,0.9718,1.0000,0.9718,0.9780,0.9712,0.9573,0.9609,0.2830
lr,Logistic Regression,0.9618,0.9974,0.9618,0.9705,0.9610,0.9422,0.9470,0.5430
ada,Ada Boost Classifier,0.9618,0.9878,0.9618,0.9705,0.9610,0.9422,0.9470,0.1280
knn,K Neighbors Classifier,0.9527,0.9888,0.9527,0.9595,0.9520,0.9284,0.9322,0.0870
xgboost,Extreme Gradient Boosting,0.9527,0.9847,0.9527,0.9636,0.9519,0.9286,0.9345,0.1250
rf,Random Forest Classifier,0.9518,0.9971,0.9518,0.9660,0.9487,0.9270,0.9352,0.2230
gbc,Gradient Boosting Classifier,0.9518,0.9855,0.9518,0.9660,0.9487,0.9270,0.9352,0.2510


...

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


Feature Importance ist für dieses Modell nicht verfügbar.


,Länge des Kelchblatts,Kelchblattweite,Blütenblattlänge,Blütenblattweite,Klasse
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


,Länge des Kelchblatts,Kelchblattweite,Blütenblattlänge,Blütenblattweite
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Transformation Pipeline and Model Successfully Saved

Das Modell wurde als 'best_model' gespeichert.


In [34]:
    # Datei-Link zum Herunterladen erstellen
link = FileLink('best_model.pkl')
display(link)

/content/best_model.pkl

In [35]:
print("Vorhersage auf trainiertes Modell prüfen:")

# Lade das trainierte Modell
loaded_model = None
try:
    loaded_model = load_model('best_model')
except:
    print("Es wurde kein Modell gefunden. Bitte trainieren Sie zuerst ein Modell.")

# Option für den Benutzer, Daten zum Vorhersagen hochzuladen
print("\nDaten für die Vorhersage hochladen oder eingeben:")

upload_button = widgets.FileUpload(description="CSV hochladen")

def on_upload_change(change):
    clear_output(wait=True)
    predict_df = pd.read_csv(upload_button.data[-1])
    # Stellen Sie sicher, dass die Ziel-Spalte (falls vorhanden) aus dem Vorhersage-DataFrame entfernt wird
    predict_df = predict_df.drop(columns=[target_column], errors='ignore')
    predictions = predict_model(loaded_model, data=predict_df)
    print("\nVorhersageergebnisse:")
    display(predictions)

upload_button.observe(on_upload_change, names='_counter')
display(upload_button)

# Lassen Sie den Benutzer die Daten manuell eingeben, jedoch ohne die Ziel-Spalte
input_data = {}
for col in df.columns:
    # Wir überspringen die Ziel-Spalte, da wir diese vorhersagen möchten
    if col != target_column:
        value = input(f"Geben Sie einen Wert für {col} ein: ")
        input_data[col] = value

# Überprüfen, ob alle Felder ausgefüllt sind
if all(val for val in input_data.values()):
    input_df = pd.DataFrame([input_data])
    predictions = predict_model(loaded_model, data=input_df)
    print(predictions.columns)
    print("\nVorhersageergebnis:")
    print(predictions['prediction_label'].iloc[0])


Vorhersage auf trainiertes Modell prüfen:
Transformation Pipeline and Model Successfully Loaded

Daten für die Vorhersage hochladen oder eingeben:


FileUpload(value={}, description='CSV hochladen')

Geben Sie einen Wert für Länge des Kelchblatts ein: 1
Geben Sie einen Wert für Kelchblattweite ein: 1
Geben Sie einen Wert für Blütenblattlänge ein: 1
Geben Sie einen Wert für Blütenblattweite ein: 1


Index(['Länge des Kelchblatts', 'Kelchblattweite', 'Blütenblattlänge',
       'Blütenblattweite', 'prediction_label', 'prediction_score'],
      dtype='object')

Vorhersageergebnis:
Iris-virginica
